# Basic Tutorial for TensorFlow


In [1]:
!pip -q install aquvitae

In [2]:
!wget -q --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1D28_TKauDVFO2nVlUeK_fuZX85zAO5o4' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1D28_TKauDVFO2nVlUeK_fuZX85zAO5o4" -O teacher.h5 && rm -rf /tmp/cookies.txt

In [3]:
import tensorflow as tf

from aquvitae import dist, ST

In [4]:
dataset = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = dataset.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

170500096/170498071 [==============================] - 2s 0us/step


In [5]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(64)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(64)

In [6]:
teacher = tf.keras.applications.ResNet50(weights=None, input_shape=(32, 32, 3), classes=10)
teacher.load_weights('teacher.h5')
student = tf.keras.applications.MobileNetV2(weights=None, input_shape=(32, 32, 3), classes=10)

In [7]:
student = dist(
    teacher=teacher,
    student=student,
    algo=ST(alpha=0.6, T=2.5),
    optimizer=tf.keras.optimizers.Adam(),
    train_ds=train_ds,
    test_ds=test_ds,
    iterations=15000
)

Training - 15000/15000 [██████████████████████████████] ELP: 09:26, accuracy: 0.5490 - val_accuracy: 0.5501 